# Gaussian1dModel Examples

### Examples taken from QuantLib's C++ Example (by Peter Caspers) translated to Python

This is some example code showing how to use the GSR (Gaussian short rate) and Markov Functional model.

In [ ]:
import QuantLib as ql

In [ ]:
refDate = ql.Date(30,4,2014)

In [ ]:
ql.Settings.instance().evaluationDate=refDate

In [ ]:
print("EvalDate: ",ql.Settings.instance().evaluationDate)

In [ ]:
forward6mLevel = 0.025
oisLevel = 0.02

In [ ]:
forward6mQuote = ql.QuoteHandle(ql.SimpleQuote(forward6mLevel))
oisQuote = ql.QuoteHandle(ql.SimpleQuote(oisLevel))

In [ ]:
yts6m = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), forward6mQuote, ql.Actual365Fixed() ))
ytsOis = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), oisQuote, ql.Actual365Fixed() ))

In [ ]:
euribor6m = ql.Euribor6M(yts6m)

We assume a multicurve setup, for simplicity with flat yield term structures. The discounting curve is an Eonia curve at a level of {{oisLevel}} and the forwarding curve is an Euribior 6m curve at a level of {{forward6mLevel}}

In [ ]:
volLevel = 0.20

In [ ]:
volQuote = ql.QuoteHandle(ql.SimpleQuote(volLevel))

In [ ]:
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed())

In [ ]:
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed()))

For the volatility we assume a flat swaption volatility at {{volLevel}}

In [ ]:
strike = 0.04

We consider a standard 10y bermudan payer swaption with yearly exercises at a strike of {{strike}}

In [ ]:
effectiveDate = ql.TARGET().advance(refDate, ql.Period(2,ql.Days))

In [ ]:
maturityDate = ql.TARGET().advance(effectiveDate, ql.Period(10,ql.Years))

In [ ]:
maturityDate

In [ ]:
fixedSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(1,ql.Years), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [ ]:
for i in fixedSchedule:
    print(i)

In [ ]:
floatingSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(6,ql.Months), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [ ]:
for i in floatingSchedule:
    print(i)

In [ ]:
underlying = ql.NonstandardSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                                              [strike]*(len(fixedSchedule)-1), ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [ ]:
for cf in underlying.fixedLeg():
    print (cf.date(), cf.amount())

In [ ]:
for cf in underlying.floatingLeg():
    print (cf.date(), cf.amount())

In [ ]:
exerciseDates = [ql.TARGET().advance(fixedSchedule[i], ql.Period(-2, ql.Days)) for i in range(1,10)]

In [ ]:
exerciseDates

In [ ]:
exercise = ql.BermudanExercise(exerciseDates, False)

In [ ]:
swaption = ql.NonstandardSwaption(underlying, exercise)

The model is a one factor Hull White model with piecewise volatility adapted to our exercise dates.

In [ ]:
stepDates = [exerciseDates[i] for i in range(0, len(exerciseDates)-1)]

In [ ]:
sigmas = [ql.QuoteHandle(ql.SimpleQuote(0.01)) for i in range(0, len(stepDates)+1)]

In [ ]:
reversion = 0.01

In [ ]:
reversions = [ql.QuoteHandle(ql.SimpleQuote(reversion))]

The reversion is just kept constant at a level of {{reversion}}

The model's curve is set to the 6m forward curve. Note that the model adapts automatically to other curves where appropriate (e.g. if an index requires a different forwarding curve) or where explicitly specified (e.g. in a swaption pricing engine).

In [ ]:
gsr = ql.Gsr(yts6m, stepDates, sigmas, reversions)

In [ ]:
swaptionEngine = ql.Gaussian1dSwaptionEngine(gsr, 64, 7.0, True, False, ytsOis, "Naive")

In [ ]:
nonstandardSwaptionEngine = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(),ytsOis, "Naive")

In [ ]:
swaption.setPricingEngine(nonstandardSwaptionEngine)

The engine can generate a calibration basket in two modes. The first one is called Naive and generates ATM swaptions adapted to the exercise dates of the swaption and its maturity date.

In [ ]:
swapBase = ql.EuriborSwapIsdaFixA(ql.Period(10,ql.Years), yts6m, ytsOis)

In [ ]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "Naive")

In [ ]:
def printBasket(basket):
    print()
    print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format("Expiry", "Maturity", "Nominal", "Rate", "Pay/Rec", "Market ivol"))
    print("===================="*4+"==================")
    for helper in basket:
        endDate = helper.swaptionMaturityDate()
        nominal = helper.swaptionNominal()
        vol = helper.volatility().value()
        rate = helper.swaptionStrike()
        expiry = helper.swaptionExpiryDate()
        swp_type = 0 # to be SWIGed
        print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format(str(expiry), str(endDate), ("%.2f"%nominal), ("%.6f"%rate), str(swp_type), ("%.6f"%vol)))

The resulting basket looks as follows:

In [ ]:
printBasket(basket)

Let's calibrate our model to this basket. We use a specialized calibration method calibrating the sigma function one by one to the calibrating vanilla swaptions. The result of this is as follows:

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [ ]:
method = ql.LevenbergMarquardt()

In [ ]:
ec = ql.EndCriteria(1000, 10, 1e-8, 1e-8, 1e-8)

In [ ]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [ ]:
def printModelCalibration(basket, volatility):
    print()
    print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format("Expiry", "Model sigma", "Model price", "Market price", "Model ivol", "Market ivol"))
    print("===================="*5)
    for i in range(0,len(basket)):
        helper = basket[i]
        expiry = helper.swaptionExpiryDate()
        print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format(str(expiry), ("%.6f"%volatility[i]), ("%.6f"%helper.modelValue()),
                                                           ("%.6f"%helper.marketValue()), ("%.6f"%helper.impliedVolatility(helper.modelValue(), 1e-6, 1000, 0.0, 2.0)),
                                                           ("%.6f"%helper.volatility().value())))
    if (len(volatility) > len(basket)):
        print('{:<20}'.format(("%.6f"%volatility[-1])))
        

In [ ]:
printModelCalibration(basket, gsr.volatility())

Finally we price our bermudan swaption in the calibrated model:

In [ ]:
print("Bermudan swaption NPV (ATM calibrated GSR) = %.6f" % swaption.NPV())

In [ ]:
probResult0 = swaption.probabilites()
print("Call probabilities (Naive):")
for prob in probResult0:
    print ("%.4f" % prob)

There is another mode to generate a calibration basket called MaturityStrikeByDeltaGamma. This means that the maturity, the strike and the nominal of the calibrating swaption are computed such that the npv and its first and second derivative with respect to the model's state variable) of the exotics underlying match with the calibrating swaption'sunderlying. Let's try this in our case.

In [ ]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [ ]:
printBasket(basket)

The calibrated nominal is close to the exotics nominal. The expiries and maturity dates of the vanillas are the same as in the case above. The difference is the strike which is now equal to the exotics strike.

Let's see how this affects the exotics npv. The recalibrated model is:

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [ ]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [ ]:
printModelCalibration(basket, gsr.volatility())

And the bermudan's price becomes:

In [ ]:
print("Bermudan swaption NPV (deal strike calibrated GSR) = %.6f" % swaption.NPV())

We can do more complicated things, let's e.g. modify the nominal schedule to be linear amortizing and see what the effect on the generated calibration basket is:

In [ ]:
nominalFixed = [ 1.0 - float(i) / (len(fixedSchedule) - 1) for i in range(0,len(fixedSchedule)-1)]
nominalFloating = [ n for n in nominalFixed for repetitions in range(2)]
strikes = [strike]*len(nominalFixed)

In [ ]:
underlying2 = ql.NonstandardSwap(ql.VanillaSwap.Payer, nominalFixed, nominalFloating, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [ ]:
swaption2 = ql.NonstandardSwaption(underlying2, exercise)

In [ ]:
swaption2.setPricingEngine(nonstandardSwaptionEngine)

In [ ]:
basket = swaption2.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [ ]:
printBasket(basket)

The notional is weighted over the underlying exercised into and the maturity is adjusted downwards. The rate on the other hand is not affected.

You can also price exotic bond's features. If you have e.g. a bermudan callable fixed bond you can set up the call right as a swaption to enter into a one leg swap with notional reimbursement at maturity. The exercise should then be written as a rebated exercise paying the notional in case of exercise.

In [ ]:
nominalFixed2 = [1.0]*len(nominalFixed)
nominalFloating2 = [0.0]*len(nominalFloating)

In [ ]:
underlying3 = ql.NonstandardSwap(ql.VanillaSwap.Receiver, nominalFixed2, nominalFloating2, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360(),
                                False, True)

In [ ]:
exercise2 = ql.RebatedExercise(exercise, [-1.0]*len(exerciseDates), 2, ql.TARGET())

In [ ]:
swaption3 = ql.NonstandardSwaption(underlying3, exercise2)

In [ ]:
oas0 = ql.SimpleQuote(0.0)
oas100 = ql.SimpleQuote(0.01)
oas = ql.RelinkableQuoteHandle(oas0)

In [ ]:
nonstandardSwaptionEngine2 = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, oas) #change discounting to 6m

In [ ]:
swaption3.setPricingEngine(nonstandardSwaptionEngine2)

In [ ]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

The calibration basket looks like this:

In [ ]:
printBasket(basket)

Note that nominals are not exactly 1.0 here. This is because we do our bond discounting on 6m level while the swaptions are still discounted on OIS level. (You can try this by changing the OIS level to the 6m level, which will produce nominals near 1.0). The npv of the call right is (after recalibrating the model):

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [ ]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [ ]:
print("Bond's bermudan call right npv = %.6f" % swaption3.NPV())

Up to now, no credit spread is included in the pricing. We can do so by specifying an oas in the pricing engine. Let's set the spread level to 100bp and regenerate the calibration basket.

In [ ]:
oas.linkTo(oas100) # Credit Spread 100bp

In [ ]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [ ]:
printBasket(basket)

The adjusted basket takes the credit spread into account. This is consistent to a hedge where you would have a margin on the float leg around 100bp, too.

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [ ]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [ ]:
print("Bond's bermudan call right npv (oas = 100bp) = %.6f" % swaption3.NPV())

The next instrument we look at is a CMS 10Y vs Euribor 6M swaption. The maturity is again 10 years and the option is exercisable on a yearly basis

In [ ]:
underlying4 = ql.FloatFloatSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                               swapBase, ql.Thirty360(), floatingSchedule,
                               euribor6m, ql.Actual360(), False, False,
                               [1.0]*(len(fixedSchedule)-1),[0.0]*(len(fixedSchedule)-1),[] ,[] ,[1.0]*(len(floatingSchedule)-1),
                                [0.001]*(len(floatingSchedule)-1),[],[])

In [ ]:
swaption4 = ql.FloatFloatSwaption(underlying4, exercise)

In [ ]:
floatSwaptionEngine = ql.Gaussian1dFloatFloatSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, "Naive")

In [ ]:
swaption4.setPricingEngine(floatSwaptionEngine)

Since the underlying is quite exotic already, we start with pricing this using the LinearTsrPricer for CMS coupon estimation

In [ ]:
reversionQuote=ql.QuoteHandle(ql.SimpleQuote(reversion))

In [ ]:
leg0 = underlying4.leg(0)
leg1 = underlying4.leg(1)

In [ ]:
cmsPricer = ql.LinearTsrPricer(swaptionVolHandle, reversionQuote)

In [ ]:
ql.setCouponPricer(leg0,cmsPricer)

In [ ]:
iborPricer = ql.BlackIborCouponPricer()

In [ ]:
ql.setCouponPricer(leg1,iborPricer)

In [ ]:
swapPricer = ql.DiscountingSwapEngine(ytsOis)

In [ ]:
underlying4.setPricingEngine(swapPricer)

In [ ]:
npv5 = underlying4.NPV()

In [ ]:
print("Underlying CMS Swap NPV = %.6f" % npv5)

In [ ]:
print("       CMS     Leg  NPV = %.6f" % underlying4.legNPV(0))

In [ ]:
print("       Euribor Leg  NPV = %.6f" % underlying4.legNPV(1))

We generate a naive calibration basket and calibrate the GSR model to it:

In [ ]:
basket = swaption4.calibrationBasket(swapBase, swaptionVol, "Naive");

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [ ]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [ ]:
printBasket(basket)

In [ ]:
printModelCalibration(basket, gsr.volatility())

In [ ]:
npv6 = swaption4.NPV()

The npv of the bermudan swaption is

In [ ]:
print("Float swaption NPV (GSR) = %.6f" % npv6)

In this case it is also interesting to look at the underlying swap npv in the GSR model.

In [ ]:
print("Float swap NPV (GSR) = %.6f" % swaption4.underlyingValue())

In [ ]:
probResult = swaption4.probabilites()

In [ ]:
print("Call probabilities (Naive):")
for prob in probResult:
    print ("%.4f" % prob)

In [ ]:
floatSwaptionEngine2 = ql.Gaussian1dFloatFloatSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, "Digital")

In [ ]:
swaption4.setPricingEngine(floatSwaptionEngine2)
probResult2 = swaption4.probabilites()

In [ ]:
print("Call probabilities (Digital):")
for prob in probResult2:
    print ("%.4f" % prob)

Not surprisingly, the underlying is priced differently compared to the LinearTsrPricer, since a different smile is implied by the GSR model.

This is exactly where the Markov functional model comes into play, because it can calibrate to any given underlying smile (as long as it is arbitrage free). We try this now. Of course the usual use case is not to calibrate to a flat smile as in our simple example, still it should be possible, of course...

In [ ]:
markovStepDates = exerciseDates

In [ ]:
cmsFixingDates = markovStepDates

In [ ]:
markovSigmas = [0.01 for i in range(0, len(markovStepDates)+1)]

In [ ]:
tenors = [ql.Period(10,ql.Years) for i in cmsFixingDates]

In [ ]:
markov = ql.MarkovFunctional(yts6m, reversion, markovStepDates, markovSigmas, swaptionVolHandle,
                            cmsFixingDates, tenors, swapBase, 16)

In [ ]:
swaptionEngineMarkov = ql.Gaussian1dSwaptionEngine(markov, 8, 5.0, True, False, ytsOis)

In [ ]:
floatEngineMarkov = ql.Gaussian1dFloatFloatSwaptionEngine(markov, 16, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, "Naive")

In [ ]:
swaption4.setPricingEngine(floatEngineMarkov)

In [ ]:
npv7 = swaption4.NPV()

The option npv is the markov model is:

In [ ]:
print("Float swaption NPV (Markov) = %.6f" % npv7)

This is not too far from the GSR price.

More interesting is the question how well the Markov model did its job to match our input smile. For this we look at the underlying npv under the Markov model

In [ ]:
print("Float swap NPV (Markov) = %.6f" % swaption4.underlyingValue())

In [ ]:
probResult3 = swaption4.probabilites()
print("Call probabilities (Digital):")
for prob in probResult3:
    print ("%.4f" % prob)

This is closer to our terminal swap rate model price. A perfect match is not expected anyway, because the dynamics of the underlying rate in the linear model is different from the Markov model, of course.

The Markov model can not only calibrate to the underlying smile, but has at the same time a sigma function (similar to the GSR model) which can be used to calibrate to a second instrument set. We do this here to calibrate to our coterminal ATM swaptions from above.

This is a computationally demanding task, so depending on your machine, this may take a while now...

In [ ]:
l=ql.FixedRateLeg(fixedSchedule, ql.Actual360(),[1.0],[0.01])

In [ ]:
for helper in basket:
    helper.setPricingEngine(swaptionEngineMarkov)

In [ ]:
markov.calibrate(basket, method, ec)

In [ ]:
printModelCalibration(basket, markov.volatility())

Now let's have a look again at the underlying pricing. It shouldn't have changed much, because the underlying smile is still matched.

In [ ]:
npv8 = swaption4.underlyingValue()

In [ ]:
print("Float swap NPV (Markov) = %.6f" % npv8)

This is close to the previous value as expected.

As a final remark we note that the calibration to coterminal swaptions is not particularly reasonable here, because the european call rights are not well represented by these swaptions. Secondly, our CMS swaption is sensitive to the correlation between the 10y swap rate and the Euribor 6M rate. Since the Markov model is one factor by construction.